In [2]:
import numpy as np 
import pandas as pd
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
def sigmoid(z):
    return 1.0  / (1.0 + np.exp(-z))


class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]

    def forward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a
    
    def evaluate(self, test_data):
        print("eval score")
    
    def backprop(self, x, y):
        del_b = [np.zeros(b.shape) for b in self.biases]
        del_w = [np.zeros(w.shape) for w in self.weights]

        # feedforward
        activation = x
        activations = [x]
        zs = [] # list of all the z vectors, layer by layer

        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z) # activation value after passing through funciton 
            activations.append(activation) # appending after z value to activations

        # backward pass
        delta = self.cost_derivative(activations[-1], y) * sigmoid_primes(zs[-1])
        del_b[-1] = delta
        del_w[-1] = np.dot(delta, activations[-2].transpose())

        # change in weights and biases
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            del_b[-l] = delta
            del_w[-l] = np.dot(delta, activations[-l-1].transpose())

        return (del_b, del_w)
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        # trian the neural network using mini-batch gradient descent. 
        # training_data = [(x, y)]

        if test_data: 
            n_test = len(test_data)
        n = len(training_data)

        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)
            ]
            
            for mini_batch in mini_batches: 
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print(f"Epoch {j}: {self.evaluate(test_data)} / {n_test}")


    def update_mini_batch(self, mini_batch, eta):
        # Update the network's weights and biases by applying gradient descent 
        # using backpropogration to a single mini-batch.# 

        del_b = [np.zeros(b.shape) for b in self.biases]
        del_w = [np.zeros(w.shape) for w in self.weights]

        for x,y in mini_batch:
            # computing the gradient of the cost function 
            delta_del_b = delta_del_w = self.backprop(x, y)

            del_b = [nb + dnb for nb, dnb in zip(del_b, delta_del_b)]
            del_w = [nw + dnw for nw, dnw in zip(del_w, delta_del_w)]

        self.weights = [w - (eta/len(mini_batch)) * nw for w, nw in zip(self.weights, del_w)] 
        self.baises  = [b - (eta/len(mini_batch)) * nb for b, nb in zip(self.biases,  del_b)] 



